<h1> 2c. Loading large datasets progressively with the tf.data.Dataset </h1>

In this notebook, we continue reading the same small dataset, but refactor our ML pipeline in two small, but significant, ways:
<ol>
<li> Refactor the input to read data from disk progressively.
<li> Refactor the feature creation so that it is not one-to-one with inputs.
</ol>
<br/>
The Pandas function in the previous notebook first read the whole data into memory -- on a large dataset, this won't be an option.

In [2]:
import datalab.bigquery as bq
import tensorflow as tf
import numpy as np
import shutil
print(tf.__version__)

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.8.0


<h2> 1. Refactor the input </h2>

Read data created in Lab1a, but this time make it more general, so that we can later handle large datasets. We use the Dataset API for this. It ensures that, as data gets delivered to the model in mini-batches, it is loaded from disk only when needed.

In [6]:
CSV_COLUMNS = ['fare_amount', 'pickuplon','pickuplat','dropofflon','dropofflat','passengers', 'key']
DEFAULTS = [[0.0], [-74.0], [40.0], [-74.0], [40.7], [1.0], ['nokey']]

# TODO: Create an appropriate input function read_dataset
def read_dataset(filename, mode, batch_size = 512):
    #TODO Add CSV decoder function and dataset creation and methods
    def decode_csv(row):
        columns = tf.decode_csv(row, record_defaults = DEFAULTS)
        features = dict(zip(CSV_COLUMNS, columns))
        label = features.pop('fare_amount')
        return features, label
      
    ## read list of file names:
    filenames_dataset = tf.data.Dataset.list_files(filename, shuffle = False)
    ## read lines from this dataset:
    textlines_dataset = filenames_dataset.flat_map(
        tf.data.TextLineDataset
    )
    ## decode each line:
    dataset = textlines_dataset.map(decode_csv)
    
    # Note:
    # use tf.data.Dataset.flat_map to apply one to many transformations (here: filename -> text lines)
    # use tf.data.Dataset.map      to apply one to one  transformations (here: text line -> feature list)

    if mode == tf.estimator.ModeKeys.TRAIN:
        num_epochs = None ## loop indefinitely
        dataset = dataset.shuffle(buffer_size = 10 * batch_size, seed = 2) ## and shuffle for training
    else:
        num_epochs = 1 ## only loop once through dataset
    
    ## repeat data set as needed (training) and form batches of data:
    dataset = dataset.repeat(num_epochs).batch(batch_size)
    return dataset
  
def get_train_input_fn():
  return read_dataset('./taxi-train.csv', mode = tf.estimator.ModeKeys.TRAIN)

def get_valid_input_fn():
  return read_dataset('./taxi-valid.csv', mode = tf.estimator.ModeKeys.EVAL)

<h2> 2. Refactor the way features are created. </h2>

For now, pass these through (same as previous lab).  However, refactoring this way will enable us to break the one-to-one relationship between inputs and features.

In [7]:
INPUT_COLUMNS = [
    tf.feature_column.numeric_column('pickuplon'),
    tf.feature_column.numeric_column('pickuplat'),
    tf.feature_column.numeric_column('dropofflat'),
    tf.feature_column.numeric_column('dropofflon'),
    tf.feature_column.numeric_column('passengers'),
]

def add_more_features(feats):
  # Nothing to add (yet!)
  return feats

feature_cols = add_more_features(INPUT_COLUMNS)

<h2> Create and train the model </h2>

Note that we train for num_steps * batch_size examples.

In [8]:
tf.logging.set_verbosity(tf.logging.INFO)
OUTDIR = 'taxi_trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time
model = tf.estimator.LinearRegressor(
      feature_columns = feature_cols, model_dir = OUTDIR)
model.train(input_fn = get_train_input_fn, steps = 200)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_every_n_hours': 10000, '_train_distribute': None, '_session_config': None, '_save_summary_steps': 100, '_log_step_count_steps': 100, '_num_worker_replicas': 1, '_task_id': 0, '_task_type': 'worker', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f40adfdbb70>, '_evaluation_master': '', '_tf_random_seed': None, '_is_chief': True, '_service': None, '_save_checkpoints_steps': None, '_global_id_in_cluster': 0, '_master': '', '_keep_checkpoint_max': 5, '_num_ps_replicas': 0, '_save_checkpoints_secs': 600, '_model_dir': 'taxi_trained'}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into taxi_trained/model.ckpt.
INFO:tensorflow:step = 1, loss = 965

<h3> Evaluate model </h3>

As before, evaluate on the validation data.  We'll do the third refactoring (to move the evaluation into the training loop) in the next lab.

In [9]:
metrics = model.evaluate(input_fn = get_valid_input_fn, steps = None)
print('RMSE on dataset = {}'.format(np.sqrt(metrics['average_loss'])))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-01-31-14:19:59
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-01-31-14:19:59
INFO:tensorflow:Saving dict for global step 200: average_loss = 109.93547, global_step = 200, loss = 45760.64
RMSE on dataset = 10.48501205444336


## Challenge Exercise

Create a neural network that is capable of finding the volume of a cylinder given the radius of its base (r) and its height (h). Assume that the radius and height of the cylinder are both in the range 0.5 to 2.0. Unlike in the challenge exercise for b_estimator.ipynb, assume that your measurements of r, h and V are all rounded off to the nearest 0.1. Simulate the necessary training dataset. This time, you will need a lot more data to get a good predictor.

Hint (highlight to see):
<p style='color:white'>
Create random values for r and h and compute V. Then, round off r, h and V (i.e., the volume is computed from the true value of r and h; it's only your measurement that is rounded off). Your dataset will consist of the round values of r, h and V. Do this for both the training and evaluation datasets.
</p>

Now modify the "noise" so that instead of just rounding off the value, there is up to a 10% error (uniformly distributed) in the measurement followed by rounding off.

In [1]:
import datalab.bigquery as bq
import tensorflow as tf
import numpy as np
import shutil
print(tf.__version__)

import math
import pandas as pd

def gen_data(n):
  r = np.random.uniform(.5, 2, n)
  h = np.random.uniform(.5, 2, n)
  v = r ** 2 * h * math.pi
  dat = pd.DataFrame({
    'r': np.round(r, 1),
    'h': np.round(h, 1),
    'v': np.round(v, 1)
  })
  return dat


dat = gen_data(5000)
dat_eval = gen_data(1000)
dat_test = gen_data(1000)

dat.head(n = 2)

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.8.0


,h,r,v
0,1.8,0.8,3.5
1,1.0,1.7,9.1


In [2]:
## write data to file:
dat.to_csv('dat-cyl-train.csv', header = False, index = False)
dat_eval.to_csv('dat-cyl-eval.csv', header = False, index = False)
dat_test.to_csv('dat-cyl-test.csv', header = False, index = False)

In [3]:
!ls -l dat-cyl*.csv

-rw-r--r-- 1 root root 12266 Feb  1 07:45 dat-cyl-eval.csv
-rw-r--r-- 1 root root 12232 Feb  1 07:45 dat-cyl-test.csv
-rw-r--r-- 1 root root 61228 Feb  1 07:45 dat-cyl-train.csv


In [4]:
!head -n 5 dat-cyl-train.csv

1.8,0.8,3.5
1.0,1.7,9.1
0.9,1.8,9.0
1.6,0.7,2.3
1.8,1.0,5.4


In [12]:
CSV_COLUMNS = ['h', 'r', 'v']
DEFAULTS = [[0.0], [0.0], [0.0]]
LABEL_COLUMN = 'v'

## create a function to read the dataset from disk:
def read_dataset(filename, mode, batch_size = 512):
  ## Add CSV decoder function: 
  def decode_csv(row):
    columns = tf.decode_csv(row, record_defaults = DEFAULTS)
    features = dict(zip(CSV_COLUMNS, columns))
    label = features.pop(LABEL_COLUMN)
    return features, label
  
  ## create dataset:
  ## read list of file names:
  filenames_dataset = tf.data.Dataset.list_files(filename, shuffle = False)
  ## read lines from this dataset list:
  textlines_dataset = filenames_dataset.flat_map(
    tf.data.TextLineDataset   ## function that returns individual text lines of the file
  )
  ## then, decode each line:
  dataset = textlines_dataset.map(decode_csv)

  # Note:
  # use tf.data.Dataset.flat_map to apply one to many transformations (here: filename -> text lines)
  # use tf.data.Dataset.map      to apply one to one  transformations (here: text line -> feature list)

  if mode == tf.estimator.ModeKeys.TRAIN:
    num_epochs = None ## loop indefinitely
    dataset = dataset.shuffle(buffer_size = 10 * batch_size, seed = 2) ## and shuffle for training
  else:
    num_epochs = 1 ## only loop once through dataset
    
  ## repeat data set as needed (training) and form batches of data:
  dataset = dataset.repeat(num_epochs).batch(batch_size)
  return dataset

## define function that gets training input, using the read_dataset function above:

def get_train_input_fn():
  return read_dataset('./dat-cyl-train.csv', mode = tf.estimator.ModeKeys.TRAIN)

def get_valid_input_fn():
  return read_dataset('./dat-cyl-eval.csv', mode = tf.estimator.ModeKeys.EVAL)


In [13]:
## create features:
INPUT_COLUMNS = [
  tf.feature_column.numeric_column('h'),
  tf.feature_column.numeric_column('r'),
]

def add_more_features(feats):
  # Nothing to add (yet!)
  return feats

## currently, don't apply any feature transformations:
feature_cols = add_more_features(INPUT_COLUMNS)

In [19]:
## (re-)set model output directory
OUTDIR = 'cyl_trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

## create and train the model:
model = tf.estimator.DNNRegressor(
  feature_columns = feature_cols, 
  hidden_units = [4, 8, 4], 
  model_dir = OUTDIR)
model.train(input_fn = get_train_input_fn, steps = 500)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_master': '', '_num_ps_replicas': 0, '_save_checkpoints_steps': None, '_service': None, '_tf_random_seed': None, '_evaluation_master': '', '_keep_checkpoint_max': 5, '_num_worker_replicas': 1, '_model_dir': 'cyl_trained', '_train_distribute': None, '_task_id': 0, '_log_step_count_steps': 100, '_session_config': None, '_save_summary_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f2f795c8908>, '_global_id_in_cluster': 0, '_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000, '_is_chief': True, '_task_type': 'worker'}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into cyl_trained/model.ckpt.
INFO:tensorflow:loss = 39003.15, step 

In [20]:
## RMSE:
metrics = model.evaluate(input_fn = get_valid_input_fn, steps = None)
print('RMSE on dataset = {}'.format(np.sqrt(metrics['average_loss'])))


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-02-01-07:59:53
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from cyl_trained/model.ckpt-500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-02-01-07:59:54
INFO:tensorflow:Saving dict for global step 500: average_loss = 1.607494, global_step = 500, loss = 803.747
RMSE on dataset = 1.2678698301315308


In [23]:
## make prediction iterator:
pred_iter = model.predict(get_valid_input_fn)
dat_pred = pd.DataFrame(columns = ['v_true', 'v_pred'])

## predict a few values to get correlation:
for i in range(1000):
  dat_pred = dat_pred.append({
    'v_true' : dat_eval['v'][i],
    'v_pred' : next(pred_iter)['predictions'][0]
  }, ignore_index = True)
  #print(dat_eval['v'][i], next(pred_iter)['predictions'][0])
  
dat_pred.head(n = 5)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from cyl_trained/model.ckpt-500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


,v_true,v_pred
0,3.4,3.418332
1,16.8,15.616695
2,2.1,0.073160
3,12.2,12.600562
4,3.5,3.418332


In [24]:
dat_pred.corr()

,v_true,v_pred
v_true,1.00000,0.97306
v_pred,0.97306,1.00000


Copyright 2017 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License